In [1]:
import duckdb as ddb
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.gridspec as gridspec

data = ddb.read_parquet("all_reviews/all_reviews.parquet")

In [ ]:
# columns and types
for x, item in enumerate(data.columns):
    print(str(item) + ":  " + str(data.dtypes[x]))

game:  VARCHAR
author_steamid:  BIGINT
author_num_games_owned:  INTEGER
author_num_reviews:  INTEGER
author_playtime_forever:  INTEGER
author_playtime_last_two_weeks:  INTEGER
author_playtime_at_review:  BIGINT
author_last_played:  INTEGER
language:  VARCHAR
review:  VARCHAR
timestamp_created:  INTEGER
timestamp_updated:  INTEGER
voted_up:  INTEGER
votes_up:  INTEGER
votes_funny:  INTEGER
weighted_vote_score:  FLOAT
comment_count:  INTEGER
steam_purchase:  INTEGER
received_for_free:  INTEGER
written_during_early_access:  INTEGER


In [ ]:
# print(data.describe())


['game', 'author_steamid', 'author_num_games_owned', 'author_num_reviews', 'author_playtime_forever', 'author_playtime_last_two_weeks', 'author_playtime_at_review', 'author_last_played', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access']
┌─────────┬─────────────────────────────┬───────────────────────┬────────────────────────┬────────────────────┬─────────────────────────┬────────────────────────────────┬───────────────────────────┬────────────────────┬────────────┬───────────────────────────────────────────────┬────────────────────┬────────────────────┬───────────────────┬────────────────────┬────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────────────┐
│  aggr   │            game             │    author_steamid     │ author_num_games_owned │ author

In [36]:
# IDEA:
# review scores from those who got the game for free vs those who had to pay for it
# is there an average of a higher rating for free / not purchased? for what games?

# plan: 
#   gather the top 3 games the most amount of reviews
#   find out the percentage of positive vs negative reviews
#   then find out the percentages from those who puchased vs did not purchase the game (include those who got the game for free)
#   analyze which games had which results, and do some further research on why this may be
#       this could be finding the reviews with the highest helpfulness or most upvotes
#       this could also include doing some outside reseach on reddit / internet

# gather the top 3 games the most amount of reviews
# most_reviewed_games = ddb.sql(f"""
#     SELECT game, COUNT(*) as review_count
#     FROM data
#     GROUP BY game
#     ORDER BY review_count DESC
#     LIMIT 3
# """).fetchdf()

# find games with middle of the road reviews, grab the top 3 in review count
mid_reviewed_games = ddb.sql(f"""
    SELECT
        game,
        COUNT(*) AS review_count,
        AVG(CASE WHEN voted_up = 1 THEN 1.0 ELSE 0 END) * 100 AS average_positive_percentage
    FROM data
    GROUP BY game
    HAVING 
        average_positive_percentage BETWEEN 40 AND 60
        AND review_count > 100
    ORDER BY review_count DESC
    -- LIMIT 10
""").fetchdf()

print(mid_reviewed_games)

                                            game  review_count  \
0                                  Call of Duty®        451506   
1                              Battlefield™ 2042        227879   
2                          鬼谷八荒 Tale of Immortal        219306   
3                             EA SPORTS™ FIFA 23        141619   
4                     Warhammer 40,000: Darktide         77394   
...                                          ...           ...   
1277         Evil Genius 2: Oceans Campaign Pack           101   
1278                            The Walking Evil           101   
1279  Insurgency: Sandstorm - Sasquatch Gear Set           101   
1280                                  Unscripted           101   
1281                        The Palace of Unrest           101   

      average_positive_percentage  
0                       59.627779  
1                       40.113832  
2                       51.709028  
3                       55.113368  
4                       59.

In [43]:
# find out the percentages from those who puchased vs did not purchase the game (include those who got the game for free)
# need all the reviews where they purchased the game AND did not get it for free, then take the percentages of those reviews
# some columns to consider:
#   steam_purchase:  INTEGER
#   received_for_free:  INTEGER
purchased_review_percentages = ddb.sql(f"""
    SELECT
        game,
        COUNT(*) AS review_count,
        SUM(CASE WHEN voted_up = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS positive_review_percentage
    FROM data
    WHERE 
        game IN (SELECT game FROM mid_reviewed_games)
        AND steam_purchase = 1
        AND received_for_free != 1
    GROUP BY game
    HAVING review_count > 100
    ORDER BY review_count DESC
""").fetchdf()

print(purchased_review_percentages)

                                                  game  review_count  \
0                                        Call of Duty®        244781   
1                                鬼谷八荒 Tale of Immortal        185173   
2                                    Battlefield™ 2042        169993   
3                                   EA SPORTS™ FIFA 23        113922   
4                           Warhammer 40,000: Darktide         65316   
5                                             PAYDAY 3         32143   
6                                             NBA 2K22         29107   
7                              Wo Long: Fallen Dynasty         17532   
8                   Infestation: Survivor Stories 2020         17236   
9                              Need for Speed™ Unbound         15935   
10                                            Devotion         15703   
11                                           Wayfinder         15373   
12                      Monster Hunter World: Iceborne         1